# Введение в обработку естественного языка
## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

Практическое задание к уроку 8. Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
!pip install --upgrade xlrd
!pip install pymorphy2
!pip install stop-words

In [3]:
import pandas as pd
import re

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

In [5]:
df = pd.read_excel('summer_reports.xls')

df.sample(15)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [6]:
df.Content = df.Content.astype(str)

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [8]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [9]:
df['Content'] = df['Content'].apply(preprocess_text)

df.sample(5)

,Rating,Content,Date
1111,1,показывать ошибка 24,2017-08-12
1997,5,удобный приложение довольный,2017-08-11
17679,5,рука,2017-07-10
486,5,огромный,2017-08-13
2729,5,серёга,2017-08-10


In [10]:
train_df, test_val_df = train_test_split(df, test_size = 0.3, random_state = 49)
test_df, val_df = train_test_split(test_val_df, test_size = 0.5, random_state = 49)

In [11]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')
val_df.to_csv('val_df.csv')
test_val_df.to_csv('test_val_df.csv')

In [12]:
text_corpus_train = train_df['Content'].values
text_corpus_valid = val_df['Content'].values
text_corpus_test = test_df['Content'].values

In [13]:
num_classes = len(df['Rating'].unique())
num_classes

5

In [14]:
tokenizer = Tokenizer(num_words = None, 
                     filters = '#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes+1)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes+1)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes+1)

In [15]:
# RNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))

model.add(SimpleRNN(132))
model.add(Dense(132, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 132, 30)           294870    
                                                                 
 masking (Masking)           (None, 132, 30)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 132)               21516     
                                                                 
 dense (Dense)               (None, 132)               17556     
                                                                 
 dropout (Dropout)           (None, 132)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 798       
                                                                 
Total params: 334,740
Trainable params: 334,740
Non-trai

2023-05-13 14:28:14.334904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 14:28:14.335852: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 7s 258ms/step - loss: 1.1425 - accuracy: 0.6535 - val_loss: 0.8739 - val_accuracy: 0.7263
Epoch 2/10
26/26 [==============================] - 5s 201ms/step - loss: 0.8984 - accuracy: 0.7189 - val_loss: 0.7483 - val_accuracy: 0.7588
Epoch 3/10
26/26 [==============================] - 5s 207ms/step - loss: 0.7669 - accuracy: 0.7535 - val_loss: 0.6999 - val_accuracy: 0.7664
Epoch 4/10
26/26 [==============================] - 6s 233ms/step - loss: 0.6841 - accuracy: 0.7712 - val_loss: 0.6701 - val_accuracy: 0.7740
Epoch 5/10
26/26 [==============================] - 7s 286ms/step - loss: 0.6099 - accuracy: 0.7914 - val_loss: 0.6552 - val_accuracy: 0.7775
Epoch 6/10
26/26 [==============================] - 9s 363ms/step - loss: 0.5527 - accuracy: 0.8105 - val_loss: 0.6594 - val_accuracy: 0.7706
Epoch 7/10
26/26 [==============================] - 7s 281ms/step - loss: 0.4985 - accuracy: 0.8280 - val_loss: 0.6892 - val_accuracy: 0.7692


In [17]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 97ms/step - loss: 0.6721 - accuracy: 0.7744


Test score: 0.6721118688583374
Test accuracy: 0.7744433879852295


In [18]:
results = []

results.append(['RNN', score[0], score[1]])

In [19]:
# LSTM

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))
model.add(LSTM(132, recurrent_dropout = 0.2))
model.add(Dense(132, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

In [20]:
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_1 (Masking)         (None, 132, 30)           0         
                                                                 
 lstm (LSTM)                 (None, 132)               86064     
                                                                 
 dense_2 (Dense)             (None, 132)               17556     
                                                                 
 dropout_1 (Dropout)         (None, 132)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 798       
                                                                 
Total params: 399,288
Trainable params: 399,288
Non-tr

In [21]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  

history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 30s 1s/step - loss: 1.2910 - accuracy: 0.6797 - val_loss: 0.9149 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 26s 992ms/step - loss: 0.9421 - accuracy: 0.7079 - val_loss: 0.8002 - val_accuracy: 0.7574
Epoch 3/10
26/26 [==============================] - 29s 1s/step - loss: 0.7893 - accuracy: 0.7477 - val_loss: 0.6981 - val_accuracy: 0.7726
Epoch 4/10
26/26 [==============================] - 24s 908ms/step - loss: 0.6967 - accuracy: 0.7655 - val_loss: 0.6628 - val_accuracy: 0.7740
Epoch 5/10
26/26 [==============================] - 29s 1s/step - loss: 0.6273 - accuracy: 0.7805 - val_loss: 0.6496 - val_accuracy: 0.7761
Epoch 6/10
26/26 [==============================] - 27s 1s/step - loss: 0.5793 - accuracy: 0.7956 - val_loss: 0.6470 - val_accuracy: 0.7851
Epoch 7/10
26/26 [==============================] - 26s 1s/step - loss: 0.5405 - accuracy: 0.8063 - val_loss: 0.6674 - val_accuracy: 0.7733
Epoch 8/10
26/

In [22]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM', score[0], score[1]])

7/7 [==============================] - 2s 227ms/step - loss: 0.6814 - accuracy: 0.7732


Test score: 0.681400716304779
Test accuracy: 0.7731526494026184


In [23]:
# GRU

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Masking(mask_value = 0.0))
model.add(GRU(64, recurrent_dropout = 0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))
model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_2 (Masking)         (None, 132, 30)           0         
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                                 
Total params: 317,852
Trainable params: 317,852
Non-tr

In [24]:
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 15s 547ms/step - loss: 1.4299 - accuracy: 0.6770 - val_loss: 0.8902 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 12s 479ms/step - loss: 0.9002 - accuracy: 0.7053 - val_loss: 0.7618 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 12s 459ms/step - loss: 0.7867 - accuracy: 0.7281 - val_loss: 0.7170 - val_accuracy: 0.7678
Epoch 4/10
26/26 [==============================] - 14s 527ms/step - loss: 0.7114 - accuracy: 0.7665 - val_loss: 0.6875 - val_accuracy: 0.7643
Epoch 5/10
26/26 [==============================] - 13s 488ms/step - loss: 0.6455 - accuracy: 0.7813 - val_loss: 0.6702 - val_accuracy: 0.7699
Epoch 6/10
26/26 [==============================] - 12s 469ms/step - loss: 0.5971 - accuracy: 0.7938 - val_loss: 0.6696 - val_accuracy: 0.7740
Epoch 7/10
26/26 [==============================] - 13s 507ms/step - loss: 0.5513 - accuracy: 0.8071 - val_loss: 0.6804 - val_accuracy: 0.7768

In [25]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU', score[0], score[1]])

7/7 [==============================] - 1s 123ms/step - loss: 0.6920 - accuracy: 0.7638


Test score: 0.6919934749603271
Test accuracy: 0.7637947797775269


In [26]:
# CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation = 'softmax'))

model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 132, 30)           294870    
                                                                 
 conv1d (Conv1D)             (None, 130, 128)          11648     
                                                                 
 activation (Activation)     (None, 130, 128)          0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                      

In [27]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 2s 70ms/step - loss: 1.5687 - accuracy: 0.5688 - val_loss: 1.1902 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 1s 43ms/step - loss: 1.2654 - accuracy: 0.5891 - val_loss: 0.9632 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 1s 47ms/step - loss: 1.0578 - accuracy: 0.6155 - val_loss: 0.7461 - val_accuracy: 0.7636
Epoch 4/10
26/26 [==============================] - 1s 46ms/step - loss: 0.9104 - accuracy: 0.6598 - val_loss: 0.6894 - val_accuracy: 0.7768
Epoch 5/10
26/26 [==============================] - 1s 54ms/step - loss: 0.8300 - accuracy: 0.6956 - val_loss: 0.6675 - val_accuracy: 0.7823
Epoch 6/10
26/26 [==============================] - 2s 61ms/step - loss: 0.7910 - accuracy: 0.7339 - val_loss: 0.6610 - val_accuracy: 0.7816
Epoch 7/10
26/26 [==============================] - 2s 64ms/step - loss: 0.7494 - accuracy: 0.7523 - val_loss: 0.6580 - val_accuracy: 0.7844
Epoch 8/10
26

In [28]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['CNN', score[0], score[1]])

7/7 [==============================] - 0s 16ms/step - loss: 0.6934 - accuracy: 0.7754


Test score: 0.693382203578949
Test accuracy: 0.7754114270210266


In [29]:
# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(SimpleRNN(132, recurrent_dropout = 0.2, return_sequences = "True"))
model.add(Conv1D(132, 3, activation = "linear"))
model.add(Conv1D(64, 1, activation = "linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation = "softmax"))      


model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(SimpleRNN(132, recurrent_dropout = 0.2, return_sequences = "True"))
model.add(Conv1D(132, 3, activation = "linear"))
model.add(Conv1D(64, 1, activation = "linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation = "softmax"))      


model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()# RNN + CNN

model = Sequential()

model.add(
    Embedding(input_dim = word_count,
              input_length = training_length,
              output_dim = 30,
              trainable = True,
              mask_zero = True))
model.add(SimpleRNN(132, recurrent_dropout = 0.2, return_sequences = "True"))
model.add(Conv1D(132, 3, activation = "linear"))
model.add(Conv1D(64, 1, activation = "linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation = "softmax"))      


model.compile(
    optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 132, 30)           294870    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 132, 132)          21516     
                                                                 
 conv1d_1 (Conv1D)           (None, 130, 132)          52404     
                                                                 
 conv1d_2 (Conv1D)           (None, 130, 64)           8512      
                                                                 
 flatten (Flatten)           (None, 8320)              0         
                                                                 
 dropout_4 (Dropout)         (None, 8320)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [30]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = 1)  


history = model.fit(X_train, y_train,
                    batch_size = 512,
                    epochs = 10,
                    verbose = 1,
                    validation_split = 0.1,
                    callbacks = [early_stopping])

Epoch 1/10
26/26 [==============================] - 13s 486ms/step - loss: 1.1195 - accuracy: 0.6893 - val_loss: 0.7964 - val_accuracy: 0.7457
Epoch 2/10
26/26 [==============================] - 13s 491ms/step - loss: 0.7928 - accuracy: 0.7398 - val_loss: 0.7324 - val_accuracy: 0.7713
Epoch 3/10
26/26 [==============================] - 12s 463ms/step - loss: 0.6788 - accuracy: 0.7694 - val_loss: 0.6875 - val_accuracy: 0.7782
Epoch 4/10
26/26 [==============================] - 13s 501ms/step - loss: 0.5938 - accuracy: 0.7915 - val_loss: 0.6780 - val_accuracy: 0.7733
Epoch 5/10
26/26 [==============================] - 13s 489ms/step - loss: 0.5338 - accuracy: 0.8048 - val_loss: 0.7274 - val_accuracy: 0.7761
Epoch 6/10
26/26 [==============================] - 13s 479ms/step - loss: 0.4771 - accuracy: 0.8304 - val_loss: 0.7954 - val_accuracy: 0.7726


In [31]:
score = model.evaluate(X_valid, y_val, batch_size = 512, verbose = 1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['RNN + CNN', score[0], score[1]])

7/7 [==============================] - 1s 127ms/step - loss: 0.7115 - accuracy: 0.7680


Test score: 0.711485743522644
Test accuracy: 0.7679896950721741


In [32]:
results_df = pd.DataFrame(results, columns = ['Model', 'Test score', 'Test accuracy'])
results_df

,Model,Test score,Test accuracy
0,RNN,0.672112,0.774443
1,LSTM,0.681401,0.773153
2,GRU,0.691993,0.763795
3,CNN,0.693382,0.775411
4,RNN + CNN,0.711486,0.767990


Выводы: результаты не сильно отличаются? и однозначно сказать, какая архитектура лучше других, нельзя.